In [1]:
# Dependencies and Setups
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey
gmaps.configure(api_key=gkey)


In [2]:
#Create DataFrame from WeatherPy Output File (CSV)

cities_df = pd.read_csv("../WeatherPy/File Output/cities.csv")
cities_df.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,carnarvon,-24.87,113.63,66.200,88.0,0.0,5.70,AU,08-28-2020
1,arraial do cabo,-22.97,-42.03,78.800,65.0,20.0,6.70,BR,08-28-2020
2,busselton,-33.65,115.33,53.996,71.0,6.0,9.35,AU,08-28-2020
3,ivanteyevka,55.97,37.92,69.800,52.0,40.0,6.00,RU,08-28-2020
4,albany,42.60,-73.97,69.998,77.0,99.0,1.34,US,08-28-2020


In [3]:
#Create gmaps figure

fig = gmaps.figure()

In [4]:
#Create heatmap layer

locations = cities_df[["Lat","Lon"]]
humidity = cities_df["Humidity"]

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False,
                             max_intensity = 100, point_radius=5)

In [5]:
#Add heatmap layer

fig.add_layer(heat_layer)

In [6]:
#Display gmaps figure

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Filter cities with ideal temp, wind, and cloud conditions & create new DataFrame

ideal_cities = cities_df.loc[(cities_df["Max Temp"]<80) & (cities_df["Max Temp"]>70) & (cities_df["Wind Speed"]<10)
                                & (cities_df["Humidity"]<40) & (cities_df["Cloudiness"]<70), :]
ideal_cities_df = pd.DataFrame(ideal_cities)
ideal_cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
100,chipinge,-20.19,32.62,79.502,26.0,20.0,3.42,ZW,08-28-2020
124,castro,-24.79,-50.01,75.938,39.0,0.0,1.95,BR,08-28-2020
198,kryve ozero,47.95,30.35,76.928,35.0,30.0,5.75,UA,08-28-2020
257,saint george,37.10,-113.58,78.998,27.0,1.0,1.50,US,08-28-2020
486,shache,38.42,77.24,72.302,33.0,2.0,6.31,CN,08-28-2020
496,derzhavinsk,51.10,66.32,73.922,37.0,42.0,2.10,KZ,08-28-2020


In [8]:
#Add "Hotel Name" column

ideal_cities_df["Hotel Name"] = ""
ideal_cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
100,chipinge,-20.19,32.62,79.502,26.0,20.0,3.42,ZW,08-28-2020,
124,castro,-24.79,-50.01,75.938,39.0,0.0,1.95,BR,08-28-2020,
198,kryve ozero,47.95,30.35,76.928,35.0,30.0,5.75,UA,08-28-2020,
257,saint george,37.10,-113.58,78.998,27.0,1.0,1.50,US,08-28-2020,
486,shache,38.42,77.24,72.302,33.0,2.0,6.31,CN,08-28-2020,
496,derzhavinsk,51.10,66.32,73.922,37.0,42.0,2.10,KZ,08-28-2020,


In [13]:
#Use Google Places API to find hotels near all ideal vacation spots

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_list = []
hotel_dict = {}

for index,row in ideal_cities_df.iterrows(): 
    try:
        lat = row["Lat"]
        long = row["Lon"]
        location = f"{lat},{long}"

        params = {
            "location": location, 
            "radius": 5000,
            "type": "hotel",
            "key": gkey}

        response = requests.get(base_url, params=params).json()

        name = response["results"][0]["name"]
   
    except:
        city = row["City"]
        print(f"No hotel information available for {city}")
        name = np.NaN
    
    ideal_cities_df.loc[index,"Hotel Name"] = name

No hotel information available for castro
No hotel information available for saint george
No hotel information available for shache
No hotel information available for derzhavinsk


In [14]:
# Display ideal cities DataFrame with Hotel Names added

ideal_cities_df = ideal_cities_df.dropna(how="any")
ideal_cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
100,chipinge,-20.19,32.62,79.502,26.0,20.0,3.42,ZW,08-28-2020,Chipinge
198,kryve ozero,47.95,30.35,76.928,35.0,30.0,5.75,UA,08-28-2020,Kryve Ozero


In [15]:
#Create marker layer with info box

locations = ideal_cities_df[["Lat","Lon"]]

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in ideal_cities_df.iterrows()]

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

In [16]:
#Add markery layer to gmaps figure

fig.add_layer(marker_layer)

In [17]:
#Display gmaps figure 

fig

Figure(layout=FigureLayout(height='420px'))